In [5]:
import pandas as pd

# Data for the table
data = {
    "Model": ["Huber", "Huber", "Huber", "M-estimators", "M-estimators", "Linear Regression", "Ridge Regression", "Ridge Regression", "Ridge Regression"],
    "Parameter": ["alpha", "epsilon", "max_iter", "min_samples", "stop_probability", "fit_intercept", "alpha", "fit_intercept", "solver"],
    "Value": [1, 1.0, 100, 0.7, 0.96, True, 0.615848211066026, True, "lsqr"],
    "R^2 Score": [0.9263, 0.9263, 0.9263, 0.9240, 0.9240, 0.9183, 0.9169, 0.9169, 0.9169]
}

# Convert data to a DataFrame
df = pd.DataFrame(data)

# Save DataFrame to .xls format
file_path = "C:/Maize_Chainat_C/45Datapoint_Master/ML_analysis/8Oct_Result/hyperparameter_results.xls"
df.to_excel(file_path, index=False)

file_path


C:\Users\Nisit\AppData\Local\Temp\ipykernel_33060\799672564.py:16: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df.to_excel(file_path, index=False)


'C:/Maize_Chainat_C/45Datapoint_Master/ML_analysis/8Oct_Result/hyperparameter_results.xls'

In [1]:
import pandas as pd
from sklearn.utils import resample

# Load the dataset
#data = pd.read_csv("C:/Maize_Chainat_C/45Datapoint_Master/7FeatureEng_Augment.csv")
#data = pd.read_csv("C:/Maize_Chainat_C/45Datapoint_Master/Env_270Dataset.csv")
#data = pd.read_csv("C:/Maize_Chainat_C/45Datapoint_Master/16Env_45Point.csv")
data = pd.read_csv("C:/Maize_Chainat_C/45Datapoint_Master/ML_analysis/7Oct_270Dataset/5VI_CD_CV_16ENV_270_01.csv")



# Bootstrap sampling to augment the data
augmented_data = resample(data, replace=True, n_samples=1000, random_state=42)

# Save the augmented data to a new CSV file
augmented_data.to_csv("C:/Maize_Chainat_C/45Datapoint_Master/ML_analysis/7Oct_270Dataset/Augment5VI_CD_CV_16ENV_1000.csv", index=False)

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor, PassiveAggressiveRegressor, RANSACRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from catboost import CatBoostRegressor
# Load the dataset
#data = pd.read_csv("C:/Maize_Chainat_C/45Datapoint_Master/5VI_RawData_30Mar_Multiparameter_V8_CV_ALL.csv")

#data = pd.read_csv("C:/Maize_Chainat_C/45Datapoint_Master/ML_analysis/6Oct_Result/10Feature_augmented_dataset_01.csv")

# data = pd.read_csv("C:/Maize_Chainat_C/45Datapoint_Master/ML_analysis/7Oct_270Dataset/Augment5VI_CD_CV_16ENV_1000.csv")

data = pd.read_csv("C:\Maize_Chainat_C\45Datapoint_Master\ML_analysis\7Oct_270Dataset\5VI_CD_CV_16ENV_270_01.csv")

# Data Preprocessing
# columns_to_drop = ['id', 'Plot_Name']
# data = data.drop(columns=columns_to_drop)
# data = data.dropna()  # Handle missing values

#X = data.drop(columns=["Seed"])
#From Top Features selected by Recursive Feature Elimination (RFE) with RandomForestRegressor:
# X = data [['Cire_CV',
#     'Temperature',
#     'Wind speed',
#     'SoilHumi',
#     'K',
#     'Evapotranspiration',
#     'P']]

# Selecting features RFE
X = data[['Cigr_CD','Cire_CD', 'EVI2_CD', 'NDRE_CD', 'NDVI_CD','Cire_CV', 'NDVI_CV', 'Maize_high', 'Humidity', 'Temperature']]


# Case2 : selected by Permutation Importance
# Case3 : selected by LASSO Regression
# Case4 : selected by Recursive Feature Elimination (RFE) with RandomForestRegressor
# case5 : 


y = data['Seed']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Models Definition
models = [
    ("CatBoost", CatBoostRegressor(verbose=0, random_state=42)),
    ("Decision Tree", DecisionTreeRegressor(random_state=42)),
    ("ElasticNet", ElasticNet()),
    ("Gradient Boosting", GradientBoostingRegressor(random_state=42)),
    ("Huber", HuberRegressor(max_iter=10000)),
    ("KNN", KNeighborsRegressor()),
    ("Lasso Regression", Lasso()),
    ("Linear Regression", LinearRegression()),
    ("M-estimators", RANSACRegressor(random_state=42)),
    ("Passive Aggressive", PassiveAggressiveRegressor(random_state=42)),
    ("Random Forest", RandomForestRegressor(random_state=42)),
    ("Ridge Regression", Ridge()),
    ("SVR", SVR()),
    ("XGBoost", XGBRegressor(random_state=42))
]

# Model Evaluation
results = []

for name, model in models:
    # Using a pipeline to ensure that scaling is applied during cross-validation steps as well
    pipeline = make_pipeline(StandardScaler(), model)
    pipeline.fit(X_train, y_train)
    predictions = pipeline.predict(X_test)
    
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    r2 = r2_score(y_test, predictions)
    
    results.append([name, mae, mse, rmse, r2])

results_df = pd.DataFrame(results, columns=["Model", "MAE", "MSE", "RMSE", "R2"])
results_df

,Model,MAE,MSE,RMSE,R2
0,CatBoost,0.065000,0.013728,0.117165,0.997748
1,Decision Tree,0.023350,0.029442,0.171585,0.995171
2,ElasticNet,0.765402,1.181669,1.087046,0.806187
3,Gradient Boosting,0.171800,0.081996,0.286349,0.986551
4,Huber,0.268496,0.662484,0.813931,0.891342
5,KNN,0.290100,0.314948,0.561202,0.948343
6,Lasso Regression,0.937438,1.537310,1.239883,0.747857
7,Linear Regression,0.370194,0.604295,0.777364,0.900886
8,M-estimators,0.300065,0.664407,0.815112,0.891027
9,Passive Aggressive,0.741806,1.196716,1.093945,0.803720
